In [5]:
# Handful of configuration

import os
import boto3
import json
from sagemaker import get_execution_role, session

region= boto3.Session().region_name

sm_client = boto3.client('sagemaker')

role = get_execution_role()
print("RoleArn: {}".format(role))

# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the data is captured
bucket =  session.Session(boto3.Session()).default_bucket()
print("Demo Bucket: {}".format(bucket))
prefix = 'sagemaker/DEMO-ModelMonitor'

s3_capture_upload_path = f's3://{bucket}/{prefix}/datacapture'
s3_report_path = f's3://{bucket}/{prefix}/reports'

print(f"Capture path: {s3_capture_upload_path}")
print(f"Report path: {s3_report_path}")

RoleArn: arn:aws:iam::806174985048:role/service-role/AmazonSageMaker-ExecutionRole-20201218T151409
Demo Bucket: sagemaker-ap-northeast-2-806174985048
Capture path: s3://sagemaker-ap-northeast-2-806174985048/sagemaker/DEMO-ModelMonitor/datacapture
Report path: s3://sagemaker-ap-northeast-2-806174985048/sagemaker/DEMO-ModelMonitor/reports


In [6]:
model_file = open("model/model.tar.gz", 'rb')
train_file = open("data/train.csv", 'rb')
test_file = open("data/test.csv", 'rb')

s3_model_key = os.path.join(prefix, 'model.tar.gz')
s3_train_key = os.path.join(prefix, 'train.csv')
s3_test_key = os.path.join(prefix, 'test.csv')

boto3.Session().resource('s3').Bucket(bucket).Object(s3_model_key).upload_fileobj(model_file)
boto3.Session().resource('s3').Bucket(bucket).Object(s3_train_key).upload_fileobj(train_file)
boto3.Session().resource('s3').Bucket(bucket).Object(s3_test_key).upload_fileobj(test_file)

print("Success! You are all set to proceed.")

Success! You are all set to proceed.


In [5]:
!pygmentize Dockerfile

FROM python:3.8-slim-buster

RUN pip3 install pandas==1.1.4 numpy==1.19.4 scikit-learn==0.23.2 scipy==1.5.4 boto3==1.17.12

WORKDIR /home

COPY src/* /home/

ENTRYPOINT ["python3", "drift_detector.py"]


In [1]:
from docker_utils import build_and_push_docker_image

repository_short_name = 'custom-model-monitor'

image_name = build_and_push_docker_image(repository_short_name)

Building docker image custom-model-monitor from Dockerfile
$ docker build -t custom-model-monitor -f Dockerfile .
Sending build context to Docker daemon  38.43MB
Step 1/5 : FROM python:3.8-slim-buster
3.8-slim-buster: Pulling from library/python
f6e04ba65310: Pulling fs layer
b87859229fd4: Pulling fs layer
598a5d7238e9: Pulling fs layer
be1d88d97f44: Pulling fs layer
22a630315c5a: Pulling fs layer
22a630315c5a: Waiting
be1d88d97f44: Waiting
b87859229fd4: Verifying Checksum
b87859229fd4: Download complete
598a5d7238e9: Verifying Checksum
598a5d7238e9: Download complete
f6e04ba65310: Verifying Checksum
f6e04ba65310: Download complete
be1d88d97f44: Verifying Checksum
be1d88d97f44: Download complete
22a630315c5a: Verifying Checksum
22a630315c5a: Download complete
f6e04ba65310: Pull complete
b87859229fd4: Pull complete
598a5d7238e9: Pull complete
be1d88d97f44: Pull complete
22a630315c5a: Pull complete
Digest: sha256:03c12f7bbd977120133b73e4b3ef5c5707ca09be338156dc02306d41633db4c0
Status: Do

In [3]:
!pygmentize src/inference.py

#  Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License").
#  You may not use this file except in compliance with the License.
#  A copy of the License is located at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  or in the "license" file accompanying this file. This file is distributed
#  on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
#  express or implied. See the License for the specific language governing
#  permissions and limitations under the License.

import os
import pickle
import pathlib

from io import StringIO

import pandas as pd

import sagemaker_xgboost_container.encoder as xgb_encoders


script_path = pathlib.Path(__file__).parent.absolute()
with open(f'{script_path}/preprocess.pkl', 'rb') as f:
    preprocess = pickle.load(f) 


def input_fn(request_body, content_type):
    """
    The SageMaker XGBoost model server receives the request data body and

In [ ]:
from sagemaker.xgboost.model import XGBoostModel
from sagemaker.serializers import CSVSerializer
from sagemaker.model_monitor import DataCaptureConfig

model_url = f's3://{bucket}/{s3_model_key}'

xgb_inference_model = XGBoostModel(
    model_data=model_url,
    role=role,
    entry_point='inference.py',
    source_dir='src',
    framework_version='1.2-1',
)

data_capture_config = DataCaptureConfig(
                        enable_capture=True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path)

predictor = xgb_inference_model.deploy(
    initial_instance_count=1,
    instance_type="ml.c5.xlarge",
    serializer=CSVSerializer(),
    data_capture_config=data_capture_config)

--

In [ ]:
s3_train_path = f's3://{bucket}/{s3_train_key}'
s3_test_path = f's3://{bucket}/{s3_test_key}'
s3_result_path = f's3://{bucket}/{prefix}/result/{predictor.endpoint_name}'

sm_client.create_monitoring_schedule(
    MonitoringScheduleName=predictor.endpoint_name,
    MonitoringScheduleConfig={
        'ScheduleConfig': {
            'ScheduleExpression': 'cron(0 * ? * * *)'
        },
        'MonitoringJobDefinition': {
            'MonitoringInputs': [
                {
                    'EndpointInput': {
                        'EndpointName': predictor.endpoint_name,
                        'LocalPath': '/opt/ml/processing/endpointdata'
                    }
                },
            ],
            'MonitoringOutputConfig': {
                'MonitoringOutputs': [
                    {
                        'S3Output': {
                            'S3Uri': s3_result_path,
                            'LocalPath': '/opt/ml/processing/resultdata',
                            'S3UploadMode': 'EndOfJob'
                        }
                    },
                ]
            },
            'MonitoringResources': {
                'ClusterConfig': {
                    'InstanceCount': 1,
                    'InstanceType': 'ml.c5.xlarge',
                    'VolumeSizeInGB': 10
                }
            },
            'MonitoringAppSpecification': {
                'ImageUri': image_name,
                'ContainerArguments': [
                    '--train_s3_uri',
                    s3_train_path,
                    '--test_s3_uri',
                    s3_test_path,
                    '--target_label',
                    'income'
                ]
            },
            'StoppingCondition': {
                'MaxRuntimeInSeconds': 600
            },
            'Environment': {
                'string': 'string'
            },
            'RoleArn': role
        }
    }
)

In [ ]:
from threading import Thread
from time import time, sleep

def invoke_endpoint(ep_name, file_name, runtime_client):
    pre_time = time()
    with open(file_name) as f:
        count = len(f.read().split('\n')) - 2 # Remove EOF and header
    
    # Calculate time needed to sleep between inference calls if we need to have a constant rate of calls for 10 hours
    ten_hours_in_sec = 10*60*60
    sleep_time = ten_hours_in_sec/count
    
    with open(file_name, 'r') as f:
        next(f) # Skip header
        
        for ind, row in enumerate(f):   
            start_time = time()
            payload = row.rstrip('\n')
            response = runtime_client(data=payload)
            
            # Print every 15 minutes (900 seconds)
            if (ind+1) % int(count/ten_hours_in_sec*900) == 0:
                print(f'Finished sending {ind+1} records.')
            
            # Sleep to ensure constant rate. Time spent for inference is subtracted
            sleep(max(sleep_time - (time() - start_time), 0))
                
    print("Done!")
    
print(f"Sending test traffic to the endpoint {predictor.endpoint_name}. \nPlease wait...")

thread = Thread(target = invoke_endpoint, args=(predictor.endpoint, 'data/infer.csv', predictor.predict))
thread.start()

In [ ]:
%load_ext autoreload

%autoreload 1

import sys
from threading import Timer

sys.path.append('src')

%aimport drift_visualizer
%aimport utils

In [ ]:
def plot_accuracy():
    df = utils.construct_df_from_result(s3_result_path)
    if df is not None:    
        drift_visualizer.plot_accuracy(df)
    Timer(3600, plot_accuracy)
    
plot_accuracy()


In [ ]:
def plot_drift_score():
    df = utils.construct_df_from_result(s3_result_path) 
    if df is not None:    
        drift_visualizer.plot_drift_score(df)
    Timer(3600, plot_drift_score)
    
plot_drift_score()

In [ ]:
def plot_p_values():
    df = utils.construct_df_from_result(s3_result_path)   
    if df is not None:            
        drift_visualizer.plot_p_values(df)
    Timer(3600, plot_p_values)
    
plot_p_values()